Buteo_jamaicensis_filtered_images: 4952 images
Ardea_herodias_filtered_images: 4929 images
Coragyps_atratus_filtered_images: 4929 images
Parus_major_filtered_images: 4890 images
Bubo_virginianus_filtered_images: 4823 images
Branta_canadensis_filtered_images: 4821 images
Corvus_cornix_filtered_images: 4821 images
Turdus_merula_filtered_images: 4691 images
Anas_platyrhynchos_filtered_images: 4081 images
Cardinalis_cardinalis_filtered_images: 3990 images

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2)
        self.conv2 = nn.Conv2d(64, 192, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(192, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(256, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.fc1 = nn.Linear(256 * 2 * 2, 4096)  
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, num_classes)
    
    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = self.pool(F.relu(self.conv5(x)))

        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

model = AlexNet(num_classes=10)  

In [2]:
from torchmetrics.classification import Accuracy
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,Subset,random_split
import random

data_path = r"C:\Users\wyatt\OneDrive\Documents\college\cu_boulder\sem4\deep_learning\Final\all_filtered_species\filtered_species"
batch_size = 64

transform = transforms.Compose([
    transforms.CenterCrop((256, 256)),  #Resize to minimum of all sizes - Will update size in cnn architecture
    transforms.RandomHorizontalFlip(p = 0.25),
    transforms.RandomRotation(degrees = 30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

data = datasets.ImageFolder(root=data_path, transform=transform)


subset_size = 5000
indices = random.sample(range(len(data)), subset_size)
data_subset = Subset(data, indices)

#change data_subset <-> data for train from whole data
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
training, testing = random_split(data, [train_size, test_size], generator=torch.Generator().manual_seed(1111))

training_dataset = DataLoader(training, batch_size=batch_size, shuffle=True, num_workers=6, pin_memory=True)
testing_dataset = DataLoader(testing, batch_size=batch_size, shuffle=True, num_workers=6, pin_memory=True)

c:\Users\wyatt\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def train_model(model, dataloader, criterion, optimizer_metric, accuracy_metric, device):
    model.train()
    net_loss = 0
    for images, labels in tqdm(dataloader, desc = "TRAINIGN"):
        images, labels = images.to(device), labels.to(device)
        y_hat = model(images)
        loss = criterion(y_hat, labels)
        optimizer_metric.zero_grad()
        loss.backward()
        optimizer_metric.step()
        accuracy_metric.update(y_hat, labels)
        net_loss += loss.item()
    
    epoch_accuracy = accuracy_metric.compute().item()
    epoch_loss = net_loss/(len(dataloader))

    return epoch_accuracy, epoch_loss

def test_model(model, dataloader, criterion, accuracy_metric, device):
    model.eval()
    net_loss = 0
    for images, labels in tqdm(dataloader, desc = "TESTING"):
        image, label = images.to(device), labels.to(device)
        y_hat = model(image)
        loss = criterion(y_hat, label)
        accuracy_metric.update(y_hat, label)
        net_loss += loss.item()
    
    epoch_accuracy = accuracy_metric.compute().item()
    epoch_loss = net_loss/(len(dataloader))

    return epoch_accuracy, epoch_loss

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

AlexNet = AlexNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(AlexNet.parameters(), lr=3e-4)
accuracy_score = Accuracy(task = 'multiclass', num_classes = 10).to(device)


patience = 3
min_delta = 3e-3
best_accuracy = 0
counter = 0
best_test_acc = 0

loss_scores_train_alexnet = []
accuracy_scores_train_alexnet = []

loss_scores_test_alexnet = []
accuracy_scores_test_alexnet = []

for epoch in range(50):  
    print(f"\nEpoch {epoch+1}")
    train_acc, train_loss = train_model(AlexNet, training_dataset, criterion, optimizer, accuracy_score, device)
    test_acc, test_loss = test_model(AlexNet, testing_dataset, criterion, accuracy_score, device)
    
    accuracy_scores_train_alexnet.append(train_acc)
    loss_scores_train_alexnet.append(train_loss)

    accuracy_scores_test_alexnet.append(test_acc)
    loss_scores_test_alexnet.append(test_loss)
    print(f"Train Acc: {train_acc:.4f} | Train Loss: {train_loss:.4f} | Test Acc: {test_acc:.4f} | Test Loss: {test_loss}")
    
    if test_acc - best_test_acc > min_delta:
        best_test_acc = test_acc
        counter = 0
    else:
        counter += 1
        print(f"No improvement. Early stopping counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break


Epoch 1


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.56it/s]


Train Acc: 0.2443 | Train Loss: 2.0336 | Test Acc: 0.2578 | Test Loss: 1.8945759891652736

Epoch 2


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.55it/s]


Train Acc: 0.3115 | Train Loss: 1.7556 | Test Acc: 0.3246 | Test Loss: 1.6170103047169795

Epoch 3


...

Train Acc: 0.7308 | Train Loss: 0.3014 | Test Acc: 0.7305 | Test Loss: 1.6065117250494405

Epoch 49


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.20it/s]


Train Acc: 0.7334 | Train Loss: 0.2853 | Test Acc: 0.7331 | Test Loss: 1.6603646172958166
No improvement. Early stopping counter: 1/3

Epoch 50


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.23it/s]

Train Acc: 0.7358 | Train Loss: 0.2881 | Test Acc: 0.7355 | Test Loss: 1.673222303390503


In [5]:
loss_scores_test_alexnet

[1.8945759891652736,
 1.6170103047169795,
 1.4350029586934718,
 1.3271928685052055,
 1.2866767344831609,
 1.2006987033247136,
 1.1643989588938604,
 1.1477025727025505,
 1.1256679791171535,
 1.097668153088109,
 1.10859398979719,
 1.0845463077227275,
 1.0746503293919725,
 1.1160799321674166,
 1.0885768257030823,
 1.0799460869257143,
 1.0479498627234478,
 1.0509607937871193,
 1.0381298835585717,
 1.1004392565513144,
 1.0683753239054259,
 1.0966268425085106,
 1.1026622709773837,
 1.135600283032372,
 1.1357563774601942,
 1.2370688238111482,
 1.1812389582192817,
 1.1904231200412827,
 1.1397881467326159,
 1.2732083473075815,
 1.2319260146342168,
 1.2205249946944567,
 1.2851149026228457,
 1.3053974347049688,
 1.3417779073423268,
 1.4233562913881679,
 1.4054155264581953,
 1.3720272349662521,
 1.4317130592404579,
 1.4590062754494804,
 1.39139675323655,
 1.5241356739381544,
 1.572923402397,
 1.5015386152429646,
 1.5165250832531727,
 1.5544276111790922,
 1.5399673305401185,
 1.6065117250494405,
 1

In [7]:
import pandas as pd

In [14]:
cnn_df = pd.read_csv('CNN_Testing.csv', index_col= 0)
cols = ['loss_alexnet_train', 'loss_alexnet_test', 'acc_alexnet_train', 'acc_alexnet_test']
data = [loss_scores_train_alexnet, loss_scores_test_alexnet, accuracy_scores_train_alexnet, accuracy_scores_test_alexnet]
alexnet_results = pd.DataFrame(list(zip(*data)), columns=cols)
alexnet_results.to_csv('AlexNet_results.csv')

In [17]:
from vit_pytorch import ViT

class BirdClassifier(nn.Module):
    def __init__(self,
                 cnn_state=False, # Whether to use CNN before ViT
                 image_size=256,
                 patch_size=16,
                 num_class=10,
                 dim=256,
                 layer_count=1,
                 head_count=1,
                 transformer_ff_neurons=256,
                 transformer_dropout=0.2):
        super().__init__()
        
        self.cnn_state = cnn_state
        self.image_size = image_size if not cnn_state else 64  # Will update if CNN is used

        if cnn_state:
            self.cnn = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),  # AlexNet
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2),
                
                nn.Conv2d(96, 256, kernel_size=5, padding=2),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2),

                nn.Conv2d(256, 384, kernel_size=3, padding=1),
                nn.ReLU(),

                nn.Conv2d(384, 384, kernel_size=3, padding=1),
                nn.ReLU(),

                nn.Conv2d(384, 256, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveAvgPool2d((self.image_size, self.image_size))  # Resize to ViT input
            )

        self.vision_transformer = ViT(
            image_size=self.image_size,
            patch_size=patch_size,
            num_classes=num_class,
            dim=dim,
            depth=layer_count,
            heads=head_count,
            mlp_dim=transformer_ff_neurons,
            dropout=transformer_dropout,
            emb_dropout=transformer_dropout,
            channels=3
        )

    def forward(self, x):
        if self.cnn_state:
            x = self.cnn(x)
        x = self.vision_transformer(x)
        return x


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = BirdClassifier(cnn_state=False, head_count=4, layer_count=4).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=3e-4)
accuracy_score = Accuracy(task = 'multiclass', num_classes = 10).to(device)

scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

patience = 3
min_delta = 3e-3
best_accuracy = 0
counter = 0
best_test_acc = 0

loss_scores_train = []
accuracy_scores_train = []
loss_scores_test = []
accuracy_scores_test = []

for epoch in range(50):  
    print(f"\nEpoch {epoch+1}")
    train_acc, train_loss = train_model(model, training_dataset, criterion, optimizer, accuracy_score, device)
    test_acc, test_loss = test_model(model, testing_dataset, criterion, accuracy_score, device)
    
    accuracy_scores_train.append(train_acc)
    loss_scores_train.append(train_loss)

    accuracy_scores_test.append(test_acc)
    loss_scores_test.append(test_loss)
    print(f"Train Acc: {train_acc:.4f} | Train Loss: {train_loss:.4f} | Test Acc: {test_acc:.4f} | Test Loss: {test_loss}")
    scheduler.step()
    
    if test_acc - best_test_acc > min_delta:
        best_test_acc = test_acc
        counter = 0
    else:
        counter += 1
        print(f"No improvement. Early stopping counter: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered.")
            break


Epoch 1


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.21it/s]


Train Acc: 0.2968 | Train Loss: 1.9640 | Test Acc: 0.3102 | Test Loss: 1.7833688737583808

Epoch 2


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.47it/s]


Train Acc: 0.3352 | Train Loss: 1.7774 | Test Acc: 0.3416 | Test Loss: 1.7222699162100448

Epoch 3


...

Train Acc: 0.5303 | Train Loss: 1.1728 | Test Acc: 0.5305 | Test Loss: 1.307984742177587

Epoch 49


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.57it/s]


Train Acc: 0.5316 | Train Loss: 1.1692 | Test Acc: 0.5317 | Test Loss: 1.313655751903041
No improvement. Early stopping counter: 1/3

Epoch 50


TESTING: 100%|██████████| 147/147 [00:17<00:00,  8.33it/s]

Train Acc: 0.5327 | Train Loss: 1.1670 | Test Acc: 0.5329 | Test Loss: 1.2986580194259176
No improvement. Early stopping counter: 2/3


In [21]:
cols = ['loss_optimize_vit_train', 'loss_optimize_vit_test', 'acc_optimize_vit_train', 'acc_optimize_vit_test']
data = [loss_scores_train, loss_scores_test, accuracy_scores_train, accuracy_scores_test]
optimize_vit = pd.DataFrame(list(zip(*data)), columns=cols)
optimize_vit.to_csv('Optimize_vit.csv')